In [1]:
%pip install langchain llama-cpp-python 
#CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import os

In [14]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
# DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information"
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are a chatbot that helps people learn a new language. You always give the answer to the question in a formal manner.
If you don't know the answer to a question, you tell them truthfully that you don't know and don't give false information. You are a helpful, respectful and honest assistant.
Your answers should not contain harmful, unethical, racist, sexist, toxic, dangerous or illegal content. Please make sure that your answers are socially unbiased and positive.
If a question does not make sense or is not factually coherent, please explain why, rather than answering something incorrectly.<</SYS>>
"""
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT +E_SYS

In [34]:
SYSTEM_PROMPT_2 = """
<<SYS>>
You are a chatbot that always gives the answer to the question in a formal, short and very simple manner. You leave out unnecessary words and if the answer is only one word you don't put a dot and the end. If you are asked to translate something you translate it.
If you don't know the answer to a question, you tell them truthfully that you don't know and don't give false information. You are a helpful, respectful and honest assistant.
Your answers should not contain harmful, unethical, racist, sexist, toxic, dangerous or illegal content. Please make sure that your answers are socially unbiased and positive.
If a question does not make sense or is not factually coherent, please explain why, rather than answering something incorrectly.<</SYS>>
"""
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT_2 +E_SYS

In [ ]:
chat_history = []

In [25]:
def get_prompt(instruction):
    return B_INST + SYSTEM_PROMPT + instruction + E_INST

os.listdir("/Users/josi/Llama2_weights")

['llama-7b.ggmlv3.q4_K_M.bin',
 '.DS_Store',
 'em_german_leo_mistral.Q5_K_M.gguf',
 'llama-2-7b.Q4_K_M.gguf',
 'llama-2-7b-chat.Q5_K_M.gguf']

In [21]:
llm = LlamaCpp(
    #model_path="/Users/josi/Llama2_weights/llama-2-7b.Q4_K_M.gguf?download=true",
    model_path = "/Users/josi/Llama2_weights/llama-2-7b-chat.Q5_K_M.gguf",
    temperature=0.75,
    max_tokens=2048,
    top_p=1,
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/josi/Llama2_weights/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  4096,  40

In [35]:
def evaluate_model(model, questions):
    correct_answers = 0
    total_questions = len(questions)

    for question, expected_answer in questions.items():
        # Get the model's answer for the current question
        prompt = get_prompt(question)
        model_answer = model(prompt)

        # Compare the model's answer to the expected answer (case-insensitive, whitespace-insensitive)
        if model_answer.strip().lower() == expected_answer.strip().lower():
            correct_answers += 1

        # Check if any of the expected words are present in the model's answer
        #if any(word.lower() in model_answer.lower() for word in expected_answer.split()):
        #    correct_answers += 1

        # Print the results for each question
        print(f"Question: {question}")
        print(f"Expected Answer: {expected_answer}")
        print(f"Model Answer: {model_answer}")
        print("------------")

    # Print overall evaluation results
    print(f"Total Questions: {total_questions}")
    print(f"Correct Answers: {correct_answers}")
    print(f"Accuracy: {correct_answers / total_questions * 100:.2f}%")


# Dictionary with questions
test_questions = {
    "What is the capital of France?": "The capital of France is Paris.",
    "Translate 'hello' to Spanish.": "Hola",
    "Write an informal greeting.": "Hey there!",
    "Explain the concept of gravity.": "Gravity is a force that attracts two objects with mass.",
    "What is the square root of 25?": "The square root of 25 is 5.",
    "Translate 'good morning' to French.": "Bonjour.",
    "Translate 'Thank you' to German.": "Danke",
    "How do you say 'hello' in Japanese?": "Konnichiwa (こんにちは)",
    "Provide the Italian translation for 'book'.": "Libro",
    "Write an informal response to the question: 'How's it going?'": "Hey! It's going well, thanks for asking!",
    "Explain the concept of artificial intelligence in an academic style.": "Artificial Intelligence (AI) refers to the development of algorithms and computational systems that enable machines to exhibit intelligent behavior, learn from experience, and perform tasks traditionally requiring human cognitive abilities. It encompasses a broad range of techniques, including machine learning, natural language processing, and expert systems. In an academic context, AI research focuses on understanding, modeling, and implementing intelligent agents capable of reasoning, problem-solving, and adapting to complex environments. The goal is to create systems that emulate human-like intelligence, allowing machines to autonomously analyze data, make informed decisions, and continuously improve their performance across diverse domains.",
    "Write a casual message inviting someone to an event.": "Hey, want to join us for an event this weekend?",
}


In [36]:
evaluate_model(llm, test_questions)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =       2.82 ms /     9 runs   (    0.31 ms per token,  3188.10 tokens per second)
llama_print_timings: prompt eval time =   26839.86 ms /   166 tokens (  161.69 ms per token,     6.18 tokens per second)
llama_print_timings:        eval time =    1542.18 ms /     8 runs   (  192.77 ms per token,     5.19 tokens per second)
llama_print_timings:       total time =   28482.26 ms
Llama.generate: prefix-match hit


Question: What is the capital of France?
Expected Answer: The capital of France is Paris.
Model Answer:   The capital of France is Paris.
------------
Question: Translate 'hello' to Spanish.
Expected Answer: Hola
Model Answer:   Hello! The translation of "hello" in Spanish is "hola."
------------



llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =       4.99 ms /    17 runs   (    0.29 ms per token,  3407.50 tokens per second)
llama_print_timings: prompt eval time =    1907.98 ms /    11 tokens (  173.45 ms per token,     5.77 tokens per second)
llama_print_timings:        eval time =    3940.34 ms /    16 runs   (  246.27 ms per token,     4.06 tokens per second)
llama_print_timings:       total time =    5925.90 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =       1.20 ms /     4 runs   (    0.30 ms per token,  3325.02 tokens per second)
llama_print_timings: prompt eval time =    1796.02 ms /    10 tokens (  179.60 ms per token,     5.57 tokens per second)
llama_print_timings:        eval time =     707.89 ms /     3 runs   (  235.96 ms per token,     4.24 tokens per second)
llama_print_timings:       total time =    2524.82 ms
Llama.generate: prefix-

Question: Write an informal greeting.
Expected Answer: Hey there!
Model Answer:   Hello!
------------
Question: Explain the concept of gravity.
Expected Answer: Gravity is a force that attracts two objects with mass.
Model Answer:   Gravity is a fundamental force that governs the behavior of objects on Earth and in the universe. It is the attractive force between two objects with mass, and it always acts towards the center of mass of each object. The strength of the gravitational force depends on the mass of the objects and the distance between them. On Earth, gravity keeps us and everything around us grounded, and it also plays a crucial role in the orbits of celestial bodies like planets and moons.
------------



llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =      29.74 ms /   104 runs   (    0.29 ms per token,  3497.44 tokens per second)
llama_print_timings: prompt eval time =    1914.66 ms /    10 tokens (  191.47 ms per token,     5.22 tokens per second)
llama_print_timings:        eval time =   18259.39 ms /   103 runs   (  177.28 ms per token,     5.64 tokens per second)
llama_print_timings:       total time =   20569.57 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =       4.87 ms /    19 runs   (    0.26 ms per token,  3898.24 tokens per second)
llama_print_timings: prompt eval time =    2426.54 ms /    20 tokens (  121.33 ms per token,     8.24 tokens per second)
llama_print_timings:        eval time =    2775.94 ms /    18 runs   (  154.22 ms per token,     6.48 tokens per second)
llama_print_timings:       total time =    5274.91 ms
Llama.generate: prefix-match hit


Question: Explain the concept of artificial intelligence in an academic style.
Expected Answer: Artificial Intelligence (AI) refers to the development of algorithms and computational systems that enable machines to exhibit intelligent behavior, learn from experience, and perform tasks traditionally requiring human cognitive abilities. It encompasses a broad range of techniques, including machine learning, natural language processing, and expert systems. In an academic context, AI research focuses on understanding, modeling, and implementing intelligent agents capable of reasoning, problem-solving, and adapting to complex environments. The goal is to create systems that emulate human-like intelligence, allowing machines to autonomously analyze data, make informed decisions, and continuously improve their performance across diverse domains.
Model Answer:   Artificial intelligence (AI) refers to the simulation of human intellectual functions by machines, particularly computers. It involve


llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =      80.91 ms /   299 runs   (    0.27 ms per token,  3695.51 tokens per second)
llama_print_timings: prompt eval time =    1772.40 ms /    15 tokens (  118.16 ms per token,     8.46 tokens per second)
llama_print_timings:        eval time =   56219.74 ms /   298 runs   (  188.66 ms per token,     5.30 tokens per second)
llama_print_timings:       total time =   59217.01 ms
Llama.generate: prefix-match hit


Question: Write a casual message inviting someone to an event.
Expected Answer: Hey, want to join us for an event this weekend?
Model Answer:   "Hey there! I'm excited to invite you to an event on [date and time]. It's going to be a great opportunity to catch up with old friends and make some new ones. Hope to see you there!"
------------
Total Questions: 12
Correct Answers: 3
Accuracy: 25.00%



llama_print_timings:        load time =    2245.84 ms
llama_print_timings:      sample time =      13.03 ms /    51 runs   (    0.26 ms per token,  3913.44 tokens per second)
llama_print_timings: prompt eval time =    1865.87 ms /    15 tokens (  124.39 ms per token,     8.04 tokens per second)
llama_print_timings:        eval time =    8468.47 ms /    50 runs   (  169.37 ms per token,     5.90 tokens per second)
llama_print_timings:       total time =   10514.76 ms


In [ ]:

# Sample Test Suite
test_suite = {
    "Test1": "Question 1",
    "Test2": "Question 2",
    "Test3": "Question 3",
    "Test4": "Question 4",
    "Test5": "Question 5",
}

# Sample Predictions from the Language Model
predictions = {
    "Test1": "Answer 1",
    "Test2": "Answer 2",
    "Test3": "Answer 3",
    "Test4": "Answer 4",
    "Test5": "Answer 5",
}

# Function to Calculate the Accuracy of the Model
def accuracy(test_suite, predictions):
    correct = 0
    total = 0
    for test in test_suite:
        total += 1
        if test_suite[test] == predictions[test]:
            correct += 1
    return correct / total

# Evaluate the Model's Performance
print("Accuracy: ", accuracy(test_suite, predictions))

